# Transformer麻雀AI 開発ワークフロー (状態調査機能付き)

**【前提条件】**
このNotebookを実行する前に、Canvasから提供された以下の3つの最新ファイルが、ご自身のGoogle Driveプロジェクト内の`src/transformer/`ディレクトリに正しく配置されていることを確認してください。
1. `transformer_parser.py`
2. `generate_data.py`
3. `investigate_state.py` (New!)

## 1. 環境設定

In [ ]:
!pip install mahjong
from google.colab import drive
drive.mount('/content/drive')

# ご指定のプロジェクトパスに移動します
%cd "/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI"

## 2. データセットの準備

In [ ]:
import os
import zipfile

ZIP_FILE_PATH = 'data.zip'
EXTRACT_DIR = '/content/data'

print(f"Copying '{ZIP_FILE_PATH}' to local environment...")
!cp -f "{ZIP_FILE_PATH}" /content/

print(f"Unzipping files to '{EXTRACT_DIR}'...")
if os.path.isdir(EXTRACT_DIR):
    !rm -rf {EXTRACT_DIR}
os.makedirs(EXTRACT_DIR, exist_ok=True)
with zipfile.ZipFile(f'/content/{ZIP_FILE_PATH}', 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_DIR)

NESTED_DATA_PATH = os.path.join(EXTRACT_DIR, 'data')
if os.path.isdir(NESTED_DATA_PATH):
    print(f"Correcting nested data directory...")
    !rsync -a {NESTED_DATA_PATH}/ {EXTRACT_DIR}/
    !rm -rf {NESTED_DATA_PATH}

print("\nChecking for unzipped files (showing first 5):")
!ls -l {EXTRACT_DIR} | head -n 6

## 3. (重要) 状態シミュレータの動作調査

1つの牌譜ファイルを選び、イベントごとにシミュレータが認識している全プレイヤーの手牌を詳細に出力します。
このログと実際の牌譜を見比べることで、どのタイミングで手牌の同期がズレるのかを正確に特定できます。

In [ ]:
# 調査対象のファイルを自動で1つ見つけます
test_file = !find /content/data -name "*.mjlog" -print -quit
if not test_file:
    test_file = !find /content/data -name "*.gz" -print -quit

if test_file:
    test_file_path = test_file[0]
    # 調査スクリプトを実行します
    !python -m src.transformer.investigate_state "{test_file_path}"
else:
    print("No .mjlog or .gz files found for investigation in /content/data")

## 4. 全データセットの生成
（原因が特定・修正されるまで、このステップは実行しないでください）

In [ ]:
!python -m src.transformer.generate_data

## 5. AIモデルの学習

In [ ]:
!python -m src.transformer.train_transformer

## 6. 学習済みAIによる予測

In [31]:
!python -m src.transformer.predict_transformer

2025-09-29 14:50:42.855619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759157442.881572   14376 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759157442.889391   14376 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759157442.909288   14376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759157442.909347   14376 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759157442.909358   14376 computation_placer.cc:177] computation placer alr